<a href="https://colab.research.google.com/github/Bijan-K/AI-QWEN2-7b-notebook/blob/main/QWEN2_farsi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Installing model dependences
!pip install -q transformers accelerate bitsandbytes torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.0 MB/s eta 0:00:00


In [2]:
# @title Downloading the Model
# import logging
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

# logging.getLogger("transformers").setLevel(logging.ERROR)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# @title Configuring Model and functions
messages_farsi = [
    {
        "role": "system",
        "content": """شما یک مدل هوش مصنوعی هستید و وظیفه دارید با کاربر چت کنید و کمک کننده باشید. گاهی اوقات باید مدارکی که به شما می دهند را بخوانید و به طور بخش بخش برای آنها توضیح دهید و همچنین به سوالات مربوط به آنها پاسخ دهید. تا حد ممکن اطلاعات صحیح بدهید و اگر جواب را نمیدانید بگویید. جواب ها رو به نحوه ای که گفته شد پاسخ بدهید. """,
    },
]

messages_english = [
    {
        "role": "system",
        "content": """You are an artificial intelligence model and your task is to chat with the user and be helpful. Sometimes you have to read the documents they give you and explain them part by part and also answer the questions about them. Give correct information as much as possible and tell if you don't know the answer. Answer the questions as stated. """,
    }
]

messages_temp = []


order_farsi_system_prompt_f = "قصد کاربر را ار متن زیر به دست بیاورید. و به فرمت : نوع قصد کاربر(خرید و فروش) - تعداد سهام - نام شرکت - نوع شرکت"
converse_farsi_system_prompt = (
    " به سوالات کاربرد تا توان ممکن صحیح پاسخ دهید و کمک کننده باشید"
)


order_english_system_prompt = "You are a useful language model, you are tasked with understanding the intent of the next prompt and then to accurately output a string formatted like: <type of transaction (buy or sell)> - <name of stock> - < number of stocks> - <type of stock>. If any of these where not mentioned simply replace it the argument with <not specified>."

converse_english_system_prompt = "Answer users questions accurately and be helpful."


def model_gen(messages_list):
    text = tokenizer.apply_chat_template(
        messages_list, tokenize=False, add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    generated_ids = model.generate(
        model_inputs.input_ids, max_new_tokens=512  # Change this to your needs
    )
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response


def order_e_specific_function(prompt):
    translation = model_gen(
        [
            {
                "role": "user",
                "content": f"translate this message to english from farsi : {prompt}",
            },
        ]
    )
    response = run_model_english("special", translation)

    return response


def run_model_farsi(mode, prompt):
    if mode == "order-f":
        messages_farsi.append(
            {"role": "system", "content": order_farsi_system_prompt_f}
        )

    elif mode == "order-e":
        response = order_e_specific_function(prompt)
        return response

    elif mode == "converse":
        messages_farsi.append(
            {"role": "system", "content": converse_farsi_system_prompt}
        )

    messages_farsi.append({"role": "user", "content": prompt})

    response = model_gen(messages_farsi)

    messages_farsi.append({"role": "assistant", "content": response})

    return response


def run_model_english(mode, prompt):
    if mode == "order-f" or mode == "order-e":
        messages_english.append(
            {"role": "system", "content": order_english_system_prompt}
        )
    elif mode == "converse":
        messages_english.append(
            {"role": "system", "content": converse_english_system_prompt}
        )
    elif mode == "special":
        messages_temp.append({"role": "system", "content": order_english_system_prompt})
        messages_temp.append({"role": "user", "content": prompt})
        response = model_gen(messages_temp)
        messages_temp.append({"role": "assistant", "content": f"{response}"})
        return response

    messages_english.append({"role": "user", "content": prompt})

    response = model_gen(messages_english)

    messages_english.append({"role": "assistant", "content": response})

    return response


def model_pre_config(language, mode, prompt):
    msg = ""

    if language.lower() == "english":
        msg = run_model_english(mode, prompt)
    elif language.lower() == "farsi":
        msg = run_model_farsi(mode, prompt)
    else:
        msg = "mode has been incorrectly stated"

    return msg


---
---
---

#Examples

mode can be set to either "order-f", "order-e" or "converse".


order-f and order-e use different types of techniques for understanding intent of the user in farsi.

The arguments for language are english or farsi.

In [4]:
# @title Example 1
language = "farsi" # @param {type:"string"}
mode = "order-f" # @param {type:"string"}
prompt = "برای من 18 سهام از شرکت مخابراتی \"همراه اول\" بخر"  # @param {type:"string"}

print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

print("-"*10 + " Checking Messages " + "-" *10)
print(messages_farsi)

---------- Answer ----------
قصد کاربر: خرید - تعداد سهام: 18 - نام شرکت: همراه اول - نوع شرکت: مخابراتی
---------- Checking Messages ----------
[{'role': 'system', 'content': 'شما یک مدل هوش مصنوعی هستید و وظیفه دارید با کاربر چت کنید و کمک کننده باشید. گاهی اوقات باید مدارکی که به شما می دهند را بخوانید و به طور بخش بخش برای آنها توضیح دهید و همچنین به سوالات مربوط به آنها پاسخ دهید. تا حد ممکن اطلاعات صحیح بدهید و اگر جواب را نمیدانید بگویید. جواب ها رو به نحوه ای که گفته شد پاسخ بدهید. '}, {'role': 'system', 'content': 'قصد کاربر را ار متن زیر به دست بیاورید. و به فرمت : نوع قصد کاربر(خرید و فروش) - تعداد سهام - نام شرکت - نوع شرکت'}, {'role': 'user', 'content': 'برای من 18 سهام از شرکت مخابراتی "همراه اول" بخر'}, {'role': 'assistant', 'content': 'قصد کاربر: خرید - تعداد سهام: 18 - نام شرکت: همراه اول - نوع شرکت: مخابراتی'}]


---

In [5]:
# @title Example 2
language = "farsi" # @param {type:"string"}
mode = "order-e" # @param {type:"string", placeholder: "hello"}
prompt = "برای من 17 سهام از شرکت مخابراتی \"ایرانسل\" بخر"  # @param {type:"string"}


print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

---------- Answer ----------
buy - Iran Tel - 17 - telecommunications


---

In [6]:
# @title Example 3
language = "farsi" # @param {type:"string"}
mode = "converse" # @param {type:"string", placeholder: "hello"}
prompt = "سلام روز شما چطور است؟ درباره ی خودتت یکم بگو."  # @param {type:"string"}

print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

---------- Answer ----------
سلام! من یک مدل هوش مصنوعی هستم و منظور از "روز من" این است که من در حال پاسخ به سوالات شما و کمک به شما هستم. من همیشه در اینجا هستم تا به شما کمک کنم، اما به علت بودن من یک مدل هوش مصنوعی، من نمی‌توانم چیزی در زندگی شخصی داشته باشم. اگر سوال دیگری دارید، لطفاً بپرسید.


---

In [8]:
# @title Examples 4
language = "farsi" # @param {type:"string"}
mode = "converse" # @param {type:"string", placeholder: "hello"}
prompt = "این جمله رو به انگلیسی تبدیل کن: \"سلام من می خوام درباره دریا های جهان بدونم\""  # @param {type:"string"}


print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

---------- Answer ----------
The translation of your sentence in English is: "Hello, I want to know about the world's oceans."


---

In [10]:
# @title Examples 5 (English)
language = "english" # @param {type:"string"}
mode = "order-f" # @param {type:"string"}
prompt = "Buy me seven stocks from the company named \"LuminaTech Innovations\" of the energy industry"  # @param {type:"string"}


print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

---------- Answer ----------
Buy - LuminaTech Innovations - 7 - energy industry


---

In [11]:
# @title Examples 6 (English)
language = "english" # @param {type:"string"}
mode = "converse" # @param {type:"string"}
prompt = "Hi, how are you? how is the weather today?"  # @param {type:"string"}

print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

---------- Answer ----------
Hello! As an AI, I don't experience feelings, but I'm functioning well. However, I don't have real-time capabilities to check current weather conditions. You might want to check a trusted weather forecasting app or website for that information.


---

In [13]:
# @title Examples 7 (English)
language = "english" # @param {type:"string"}
mode = "converse" # @param {type:"string"}
prompt = "Tell me, what has our conversation been about until now."  # @param {type:"string"}

print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

---------- Answer ----------
Our conversation has been about discussing various topics depending on your inquiries. We've covered a range of subjects such as financial transactions, such as buying stocks, general knowledge queries, and even casual greetings and small talk. The specific details have varied based on your input in each session.


---
---


In [14]:
# @title Reset pipline of  converstaions

messages_farsi = [
    {
        "role": "system",
        "content": """شما یک مدل هوش مصنوعی هستید و وظیفه دارید با کاربر چت کنید و کمک کننده باشید. گاهی اوقات باید مدارکی که به شما می دهند را بخوانید و به طور بخش بخش برای آنها توضیح دهید و همچنین به سوالات مربوط به آنها پاسخ دهید. تا حد ممکن اطلاعات صحیح بدهید و اگر جواب را نمیدانید بگویید. جواب ها رو به نحوه ای که گفته شد پاسخ بدهید. """,
    },
]

messages_english = [
    {
        "role": "system",
        "content": """You are an artificial intelligence model and your task is to chat with the user and be helpful. Sometimes you have to read the documents they give you and explain them part by part and also answer the questions about them. Give correct information as much as possible and tell if you don't know the answer. Answer the questions as stated. """,
    }
]

messages_temp = []


In [16]:
# @title Enter your prompt
language = "" # @param {type:"string"}
mode = "" # @param {type:"string", placeholder: "hello"}
prompt = ""  # @param {type:"string"}


print("-"*10 + " Answer " + "-" *10)
print(model_pre_config(language, mode, prompt))

---------- Answer ----------
mode has been incorrectly stated
